In [1]:
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

#데이터 가져오기

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import pandas as pd
train = pd.read_csv('/content/drive/MyDrive/악성 URL/DATA/train_FE')

In [4]:
train_df = train.loc[:,['ID','URL', 'label', 'length', 'subdomain_count', 'special_char_count',
       'number_of_meaning_words', 'tld_malicious', 'path_depth',
       'max_numeric_sequence','https', 'blacklist_word_count',
       'digit_count', 'suspicious_keyword_flag']]

In [5]:
train_df['tld_malicious'] = train_df['tld_malicious']*1
train_df['tld_malicious'].value_counts()

,count
tld_malicious,
0,6225887
1,769169


In [11]:
X,y = train_df.drop(columns=['URL','label','ID']),train_df['label']
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=42)

# 소프트 보팅 앙상블 모델

In [12]:
pip install catboost

In [13]:
from sklearn.ensemble import RandomForestClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier

clf1 = RandomForestClassifier(
    criterion='gini',
    max_depth=30,
    max_features='sqrt',
    min_samples_leaf=25,
    min_samples_split=50,
    n_estimators=200,
    random_state=42,
    n_jobs= -1
)
clf2 = LGBMClassifier(
    colsample_bytree=1.0,
    learning_rate=0.1,
    max_depth=30,
    min_child_samples=25,
    n_estimators=200,
    num_leaves=50,
    random_state=42,
    n_jobs = -1
)


clf3 = CatBoostClassifier(
    depth=8,
    learning_rate=0.05,
    l2_leaf_reg=3,
    iterations=500,
    verbose=0,
    random_state=42,
    task_type="GPU"
)


# 소프트 보팅 앙상블 모델
voting_clf = VotingClassifier(estimators=[('rf', clf1), ('lgbm', clf2), ('cat', clf3)], voting='soft')

# 모델 학습
voting_clf.fit(X_train, y_train)



[LightGBM] [Info] Number of positive: 1252583, number of negative: 4343461
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.548339 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 614
[LightGBM] [Info] Number of data points in the train set: 5596044, number of used features: 11
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.223834 -> initscore=-1.243464
[LightGBM] [Info] Start training from score -1.243464


VotingClassifier(estimators=[('rf',
                              RandomForestClassifier(max_depth=30,
                                                     min_samples_leaf=25,
                                                     min_samples_split=50,
                                                     n_estimators=200,
                                                     n_jobs=-1,
                                                     random_state=42)),
                             ('lgbm',
                              LGBMClassifier(max_depth=30, min_child_samples=25,
                                             n_estimators=200, n_jobs=-1,
                                             num_leaves=50, random_state=42)),
                             ('cat',
                              <catboost.core.CatBoostClassifier object at 0x7b979cb518d0>)],
                 voting='soft')

#roc_auc 스코어

In [14]:
from sklearn.metrics import roc_auc_score

y_pred_proba = voting_clf.predict_proba(X_test)[:,1]
roc_auc = roc_auc_score(y_test, y_pred_proba)

print(f'Soft Voting Classifier Accuracy: {roc_auc}')

Soft Voting Classifier Accuracy: 0.9303211304345309


In [16]:
# 평가 데이터 추론
import numpy as np
test_df = pd.read_csv('/content/drive/MyDrive/악성 URL/DATA/test_FE.csv')
test_X= test_df.drop(columns=['ID','URL', 'short_url'])

test_probabilities = np.zeros(len(test_X))

for model in voting_clf.estimators_:
    test_probabilities += model.predict_proba(test_X)[:, 1]


# Soft-Voting 앙상블 (Fold 별 모델들의 예측 확률 평균)
print('Inference Done.')

Inference Done.


In [17]:
test_df['probability'] = test_probabilities
test_df[['ID', 'probability']].to_csv('./submission.csv', index=False)
print('Done.')

Done.
